In [1]:
#!pip install google-api-python-client

In [2]:
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [3]:
cred=pd.read_csv('ytb_key.csv')

In [4]:
cred=cred['API_KEY'].values[0]

In [5]:
API_KEY=cred

In [6]:
def get_trending_videos(api_key, max_results=200):
    try:
        # Build the YouTube service
        youtube = build('youtube', 'v3', developerKey=api_key)

        # Initialize the list to hold video details
        videos = []

        # Fetch the most popular videos
        request = youtube.videos().list(
            part='snippet,contentDetails,statistics',
            chart='mostPopular',
            regionCode='IN',  
            maxResults=50
        )

        # Paginate through the results if max_results > 50
        while request and len(videos) < max_results:
            response = request.execute()
            for item in response['items']:
                video_details = {
                    'video_id': item['id'],
                    'title': item['snippet']['title'],
                    'description': item['snippet']['description'],
                    'published_at': item['snippet']['publishedAt'],
                    'channel_id': item['snippet']['channelId'],
                    'channel_title': item['snippet']['channelTitle'],
                    'category_id': item['snippet']['categoryId'],
                    'tags': item['snippet'].get('tags', []),
                    'duration': item['contentDetails']['duration'],
                    'definition': item['contentDetails']['definition'],
                    'caption': item['contentDetails'].get('caption', 'false'),
                    'view_count': item['statistics'].get('viewCount', 0),
                    'like_count': item['statistics'].get('likeCount', 0),
                    'dislike_count': item['statistics'].get('dislikeCount', 0),
                    'favorite_count': item['statistics'].get('favoriteCount', 0),
                    'comment_count': item['statistics'].get('commentCount', 0)
                }
                videos.append(video_details)

            # Get the next page token
            request = youtube.videos().list_next(request, response)

        return videos[:max_results]
    
    except HttpError as e:
        print(f'An HTTP error {e.resp.status} occurred: {e.content}')
        return []

In [7]:
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    return df

In [8]:
def main():
    trending_videos = get_trending_videos(API_KEY)
    if trending_videos:  # Check if any data was returned
        filename = 'trending_videos.csv'
        df=save_to_csv(trending_videos, filename)
        print(df)
        print(f'Trending videos saved to {filename}')
    else:
        print('No data to save.')

In [9]:
if __name__ == '__main__':
    main()

        video_id                                              title  \
0    2zaS5bWy5yU                 The Crazy State of INDIAN PODCASTS   
1    n4Pf4OCpRrI  Teenage Troubles ! - Hostel & Relationships | ...   
2    WR8PyAhn6tQ  Vidaamuyarchi - Pathikichu Lyric | Ajith Kumar...   
3    gzIzlhxBRU4  Try Not to Laugh or Dare Challenge vs Ruchika ...   
4    AATu-TO7trE  Yezhu Kadal Yezhu Malai - Trailer | Ram | Nivi...   
..           ...                                                ...   
116  aFAqRFO-3m4  रावण - अंगद प्रसंग | Dr Kumar Vishwas | अपने-अ...   
117  kFbKZ6Z0YPM  Miss Bezawada Mr Bhimavaram | Episode 04/06 | ...   
118  UqKwsGIbmuI  Charche - Xvir Grewal & Dhanda Nyoliwala (Musi...   
119  yvXi5HOsJKM  Auto Expo 2025 🇮🇳 - Mahindra ka Dhamaka ❤️| Wa...   
120  tfZcVmf05sk  Bigg Boss Tamil 7 | Streaming 24X7 | Now Strea...   

                                           description          published_at  \
0    We talk about how everyone has a podcast these...  2025-01-21T

In [10]:
trending_videos = pd.read_csv('trending_videos.csv')
print(trending_videos.head())

      video_id                                              title  \
0  2zaS5bWy5yU                 The Crazy State of INDIAN PODCASTS   
1  n4Pf4OCpRrI  Teenage Troubles ! - Hostel & Relationships | ...   
2  WR8PyAhn6tQ  Vidaamuyarchi - Pathikichu Lyric | Ajith Kumar...   
3  gzIzlhxBRU4  Try Not to Laugh or Dare Challenge vs Ruchika ...   
4  AATu-TO7trE  Yezhu Kadal Yezhu Malai - Trailer | Ram | Nivi...   

                                         description          published_at  \
0  We talk about how everyone has a podcast these...  2025-01-21T13:58:29Z   
1  Cross paths. Date local- https://gethap.pn/vfo...  2025-01-21T09:56:43Z   
2  Lyca Productions \nSubaskaran Presents\nAjith ...  2025-01-19T05:16:07Z   
3  Who won?\n\nInstagram ► https://www.instagram....  2025-01-19T08:30:23Z   
4  #YezhuKadalYezhuMalai #7Kadal7Malai #7Seas7Hil...  2025-01-20T12:47:47Z   

                 channel_id      channel_title  category_id  \
0  UCtgGOdTlM-NdJ9rPKIYN8UQ        Slayy Point       

In [19]:
# check for missing values
missing_values = trending_videos.isnull().sum()

# display data types
data_types = trending_videos.dtypes

missing_values, data_types

(video_id          0
 title             0
 description       0
 published_at      0
 channel_id        0
 channel_title     0
 category_id       0
 tags              0
 duration          0
 definition        0
 caption           0
 view_count        0
 like_count        0
 dislike_count     0
 favorite_count    0
 comment_count     0
 dtype: int64,
 video_id                       object
 title                          object
 description                    object
 published_at      datetime64[ns, UTC]
 channel_id                     object
 channel_title                  object
 category_id                     int64
 tags                           object
 duration                       object
 definition                     object
 caption                          bool
 view_count                      int64
 like_count                      int64
 dislike_count                   int64
 favorite_count                  int64
 comment_count                   int64
 dtype: object)

In [20]:
# fill missing descriptions with "No description"
trending_videos['description'].fillna('No description', inplace=True)

In [21]:
# convert `published_at` to datetime
trending_videos['published_at'] = pd.to_datetime(trending_videos['published_at'])

# convert tags from string representation of list to actual list
trending_videos['tags'] = trending_videos['tags'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [22]:
# descriptive statistics
descriptive_stats = trending_videos[['view_count', 'like_count', 'dislike_count', 'comment_count']].describe()

descriptive_stats

,view_count,like_count,dislike_count,comment_count
count,1.210000e+02,121.000000,121.0,121.000000
mean,2.309871e+06,65544.652893,0.0,2744.644628
std,4.629876e+06,126597.709094,0.0,5413.048203
min,4.801800e+04,0.000000,0.0,0.000000
25%,2.975690e+05,5483.000000,0.0,197.000000
50%,8.019750e+05,15789.000000,0.0,677.000000
75%,2.082636e+06,66515.000000,0.0,2964.000000
max,3.091546e+07,844789.000000,0.0,40673.000000
